City Neighborhood Classifier 

CS 305: Machine Learning Fall 2021

Sara Clark, Christine Pourheydarian, Jiawei Liu


In [13]:
!pip3 install kaggle

In [26]:
import numpy as np
import pandas as pd
import csv
import random
random.seed(42)

In [27]:
'''
importing the calendar.csv data. 
listing id stays a number
date is kept in its form but has the dashes removed
available: if f, then becomes 0. if t, then becomes 1. 
price: converted from currency to a double. if there is no price, the value inserted is None. 
'''


from re import sub
from decimal import Decimal


calendar_csvfile = open('archive/calendar.csv', 'r')#listing_id,date,available,price
reader = csv.reader(calendar_csvfile, delimiter=',', quotechar='"')
calendar_data = []

for row in reader: 
    r  = [ ]
    for i in range(4):
        if i == 0:
            date = (row[1]).replace("-", "")
            r.append(date)
        elif i == 1:
            available = 0 if row[2] == "f" else 1 #f: 0, t: 1 
            r.append(available)
        elif i == 2:        
            money = row[3]
            if money == "":
                r.append(None)
                continue
            else: 
                value = float(Decimal(sub(r'[^\d.]', '', money)))#remove dollar sign, comma, and everything after the .
                r.append(value)
        elif i == 3: 
            listing_id  = int(row[0])
            r.append(listing_id)    
    calendar_data.append(r)


calendar_data_np = np.array(calendar_data)
print(calendar_data_np)

[['20170905' 0 None 12147973]
 ['20170904' 0 None 12147973]
 ['20170903' 0 None 12147973]
 ...
 ['20160908' 0 None 14504422]
 ['20160907' 0 None 14504422]
 ['20160906' 0 None 14504422]]


In [30]:
'''
Loading reviews.cvs into numpy w/o any modification, i.e. w/o switching anything from qualitative to quantitative  
'''

csv_file = open('archive/reviews.csv', 'r')
reader = csv.reader(csv_file, delimiter=',', quotechar='"')

data = []
for i, row in enumerate(reader):
    if i == 0:
        continue
    r = []
    for i in range(6):
        r.append(row[i])
    data.append(r)

csv_file.close()
reviews = np.array(data)

['1178162' '4724140' '2013-05-21' '4298113' 'Olivier'
 "My stay at islam's place was really cool! Good location, 5min away from subway, then 10min from downtown. The room was nice, all place was clean. Islam managed pretty well our arrival, even if it was last minute ;) i do recommand this place to any airbnb user :)"]
["My stay at islam's place was really cool! Good location, 5min away from subway, then 10min from downtown. The room was nice, all place was clean. Islam managed pretty well our arrival, even if it was last minute ;) i do recommand this place to any airbnb user :)"
 'Great location for both airport and city - great amenities in the house: Plus Islam was always very helpful even though he was away'
 "We really enjoyed our stay at Islams house. From the outside the house didn't look so inviting but the inside was very nice! Even though Islam himself was not there everything was prepared for our arrival. The airport T Station is only a 5-10 min walk away. The only little is

In [17]:
# listings = np.loadtxt('archive/listings.csv', delimiter=',', skiprows = 1)
'''
but leaving out rows that dont have a value for host_neighborhood  and/or neighborhood 

'''
from re import sub
from decimal import Decimal


listings_csvfile = open('archive/listings.csv', 'r') 
reader = csv.reader(listings_csvfile, delimiter=',', quotechar='"')
listings_data = []
column = None 
property_types = {'': 0, 
              'Condominium': 1, 
              'Camper/RV': 2, 
              'House': 3, 
              'Townhouse': 4, 
              'Entire Floor': 5, 
              'Guesthouse': 6, 
              'Boat': 7, 
              'Dorm': 8, 
              'Villa': 9, 
              'Bed & Breakfast': 10, 
              'Other': 11, 
              'Apartment': 12, 
              'Loft': 13 
             }

room_types = {'Private room': 0, 'Shared room': 1, 'Entire home/apt': 2}
beds = {'Couch': 0, 'Airbed':1, 'Pull-out Sofa':2, 'Real Bed':3, 'Futon':4}
calendar_updated = {'never': 0,
                    'yesterday': 1, 
                    'today': 2,
                    '2 days ago': 3,
                    '3 days ago': 4,
                    '4 days ago': 5, 
                    '5 days ago': 6, 
                    '6 days ago': 7, 
                    'a week ago': 8,
                    '1 week ago': 9,
                    '2 weeks ago': 10, 
                    '3 weeks ago': 11,
                    '4 weeks ago': 12, 
                    '5 weeks ago': 13, 
                    '6 weeks ago': 14, 
                    '7 weeks ago': 15,
                    '2 months ago': 16,
                    '3 months ago': 17,
                    '4 months ago': 18,
                    '5 months ago': 19,
                    '7 months ago': 20,
                    '6 months ago': 21, 
                    '8 months ago': 22,
                    '9 months ago': 23,
                    '10 months ago':24,
                    '11 months ago':25,
                    '12 months ago':26, 
                    '13 months ago': 27, 
                    '14 months ago': 28, 
                    '15 months ago': 29, 
                    '16 months ago': 30, 
                    '17 months ago': 31, 
                    '18 months ago': 32, 
                    '20 months ago': 33, 
                    '22 months ago': 34, 
                    '30 months ago': 35, 
                    '23 months ago': 36, 
                    '25 months ago': 37 
             }
cancellation_policy = {'flexible': 0, 
                       'moderate': 1, 
                       'strict': 2, 
                       'super_strict_30': 3}
indicesSet = set([0,19,32,33,43,48,49,53,54,55,56,59,65, \
                  67,68,71,72,73,74,76,79,80,81,82,83,84,85,93,94,50,86,89,91,92,51,52,57,\
                 60,61,62,63,64,66,69,90])
for index, row in enumerate(reader):
        
    if index == 0:
        columns = row
    r  = [ ]
    for i in range(95):
        if i in indicesSet:
            r.append(row[i])
        elif row[i] == "N/A" or row[i] == "":
                r.append(None)
                continue 
        elif i == 0 or i == 19 or i == 32 or i == 33 or i == 43 \
        or i == 48 or i == 49 or i == 53 or i == 54 or i == 55 or i == 56 or i == 59 \
        or i == 65 or i == 67 or i == 68 or (i >=71 and i <= 74) or i == 76 or (i >= 79 \
        and i <= 85) or i ==  93 or i == 94: #listing id, host id 
            val = row[i] 
            r.append(val)
        elif i == 50 or i == 86 or i == 89 or i == 91 or i == 92:
            boolean = 0 if row[i] == "f" else 1 
            r.append(boolean)
        elif i == 51: 
            property_type = property_types[row[i]]
            r.append(property_type)
        elif i == 52:
            room_type = room_types[row[i]] 
            r.append(room_type)
        elif i == 57:
            bed = row[i]
            r.append(beds[bed])
        elif (i >=  60 and i <= 64) or i == 66: 
            money = row[i]
            if money == "":
                r.append(None)
                continue
            else: 
                value = float(Decimal(sub(r'[^\d.]', '', money)))#remove dollar sign, comma, and everything after the .
                r.append(value)
        elif i == 69: #calendar_updated col 
            date = row[i]
            r.append(calendar_updated[date])  
        elif i == 90:
            policy = row[i]
            val = cancellation_policy[(policy)] 
            r.append(val)
        
    listings_data.append(r)

# print(columns[91])

listings_numerical_data = np.array(listings_data)
print(listings_numerical_data[1])

['12147973', None, None, '31303940', None, None, None, '1', '1', None, '02131', '42.282618795779484', '-71.13306792912681', 't', 'House', 'Entire home/apt', '4', '1.5', '2', '3', 'Real Bed', '', '$250.00', '', '', '', '$35.00', '1', '$0.00', '2', '1125', '2 weeks ago', None, '0', '0', '0', '0', '0', None, None, '', '', '', '', '', '', '', 'f', None, None, 'f', 'moderate', 'f', 'f', '1', '']


<ipython-input-17-e3ecef8ceed4>:127: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  listings_numerical_data = np.array(listings_data)



Christine's to-dos:
-  remove stuff internally used
-  remove urls 
-  None for all empty values
- leave dates in date form 


Text Analysis
- Remove neighborhood names from text-based feature columns used for training
- Bag of words: tfid vectorization
- Link reviews to correct neighborhood



In [33]:
#Here is where Jiawei and Sara will do text analysis
# split data by neighborhood

def readSpamData(ham_filename, spam_filename):
    DATA = []

    # Read in ham messages from file
    ham_file = open(ham_filename, 'r')
    messages = ham_file.read().split('*****@@@@@%%%%%*****@@@@@%%%%%')
    for message in messages: DATA.append([message, 0])
    ham_file.close()
    
    # Read in spam messages from file
    spam_file = open(spam_filename, 'r')
    messages = spam_file.read().split('*****@@@@@%%%%%*****@@@@@%%%%%')
    for message in messages: DATA.append([message, 1])
    spam_file.close()

    random.shuffle(DATA)  # Shuffle
    return [row[0] for row in DATA], [row[1] for row in DATA]

# corpus, labels = readSpamData('ham.txt', 'spam.txt')
corpus = reviews[:,5]
labels = reviews[:,1]
print(len(corpus))
print(len(labels))
print(len(reviews))
# print(reviews[:,5])
# Separate into training and testing data
TEST_SIZE = 0.25  
# MAKE SURE ITS SPLIT BEFORE USING!!!!!

separator = int((1.0 - TEST_SIZE)*len(corpus))
corpus_train = corpus[:separator]
labels_train = labels[:separator]
corpus_test = corpus[separator:]
labels_test = labels[separator:]
print(len(corpus_train))
print(len(labels_train))
print(len(corpus_test))
print(len(labels_test))

68275
68275
68275
51206
51206
17069
17069


In [42]:
# Text feature extraction for training data
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()  # Use individual words as tokens
X_train = vectorizer.fit_transform(corpus_train)
y_train = np.array(labels_train)
print(X_train.shape)
print(y_train.shape)

(51206, 41894)
(51206,)
  (0, 16004)	0.18159843451113022
  (0, 33203)	0.2567278598437474
  (0, 16131)	0.18028549566396285
  (0, 35386)	0.09542250388673663
  (0, 1999)	0.24428367190544095
  (0, 25256)	0.283466098191063
  (0, 16691)	0.16718873142405255
  (0, 17256)	0.09115961611960623
  (0, 2072)	0.2673801565852983
  (0, 6697)	0.2042350834151937
  (0, 2199)	0.06847169650435873
  (0, 1678)	0.2379027799362711
  (0, 5000)	0.2555321494561598
  (0, 13797)	0.10767348112623298
  (0, 15324)	0.21356723825201737
  (0, 12394)	0.19440614825535793
  (0, 33096)	0.07198351557737255
  (0, 3592)	0.2129956590189964
  (0, 20054)	0.12628185462711838
  (0, 36110)	0.1594374640530305
  (0, 18077)	0.5064715447060048


In [35]:
# Text feature extraction for testing data
X_test = vectorizer.transform(corpus_test)
y_test = np.array(labels_test)
print(X_test.shape)
print(y_test.shape)

(17069, 41894)
(17069,)


Feature Scaling

In [ ]:
#Here is where Christine will feature scale


split into training and testing
- first organize stuff by neighborhood
- randomly permute rows of each neighborhood set
- then select 25% of each neighborhood for testing and the rest for training
- combine the testing components and the training components

- **** we also planned to do cross validation, but maybe we can revisit that later?


In [20]:
#Here is where Christine will split data and randomly permute rows

# EX. copied from previous exercise

# Split data into training (80%) and testing (20%).
# Use feature scaling to standardize the features.
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
def scaleData(trainX, testX):
    scaler = StandardScaler()
    scaler.fit(trainX)
    return scaler.transform(trainX), scaler.transform(testX)

X_train, X_test = scaleData(X_train, X_test)

NameError: name 'train_test_split' is not defined

In [ ]:
# Cross validation

Create Classification models
- kNN
- random forest
- SVM
- Logistic Regression
- Neural Network (or Perceptron)




In [41]:
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model  # Using sklearn Perceptron classifier
from sklearn import ensemble  # Using RandomForest classifier
from sklearn.svm import SVC

from sklearn import neighbors  # Using nearest neighbors classifier
# learners = {'Perceptron': linear_model.Perceptron(max_iter=10),
#             'RandomForest': ensemble.RandomForestClassifier(),
#             'kNN': neighbors.KNeighborsClassifier(),
#             'SVM': SVC(),
#             'LogisticRegression': linear_model.LogisticRegression(random_state=42)
#            },
           }
# for classifierName in learners:
#     learners[classifierName].fit(X_train, y_train)
#     print('Accuracy of ' + classifierName + ':\t' + str(learners[classifierName].score(X_test, y_test)))

Accuracy of kNN:	0.0


In [39]:
#Here is where Jiawei will do NN
# Get Perceptron weights
# learner = linear_model.Perceptron(max_iter=1)
learner.fit(X_train, y_train)
weights = learner.coef_  # Get the learned Perceptron weights
sorted_weights = np.argsort(weights)  # Sort
features = vectorizer.get_feature_names()  # Get the features
print('\nLowest weighted words (indicative of ham)')
for i in range(5): print(features[sorted_weights[0,i]])
print('\nHighest weighted words (indicative of spam)')
for i in range(5): print(features[sorted_weights[0,len(sorted_weights[0])-i-1]])

AttributeError: 'KNeighborsClassifier' object has no attribute 'coef_'